In [3]:
import numpy as np
from PIL import Image, ImageDraw, ImageFont, ImageOps
import matplotlib.pyplot as plt
import os

def load_image(image_path):
    """
    Load and preprocess image using PIL
    """
    img = Image.open(image_path)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    return np.array(img)

def create_text_mask(image_path, texts, font_size=None, save_path=None):
    """
    Create and save an inverted text mask (black text, white background)
    
    Args:
        image_path (str): Path to the input image
        texts (list): List of dictionaries containing text properties:
            [
                {
                    'text': 'Hello',            # The text to draw
                    'pos': (0.5, 0.5),         # (x, y) relative position (0-1)
                    'font_size': 100,          # Optional: specific font size
                    'rotation': 0,             # Rotation angle in degrees
                    'alignment': 'center'      # 'left', 'center', 'right'
                },
                ...
            ]
        font_size (int): Default font size if not specified in texts
        save_path (str): Path to save the mask
    
    Returns:
        numpy.ndarray: The created mask
    """
    # Load image to get dimensions
    img = Image.open(image_path)
    width, height = img.size
    
    # Create white background
    mask_img = Image.new('L', (width, height), 255)
    
    # Try to load font
    def get_font(size):
        try:
            return ImageFont.truetype("arial.ttf", size)
        except:
            try:
                return ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", size)
            except:
                print("Using default font")
                return ImageFont.load_default()
    
    # Process each text
    for text_props in texts:
        # Extract properties with defaults
        text = text_props['text']
        rel_x, rel_y = text_props.get('pos', (0.5, 0.5))  # Default to center
        rotation = text_props.get('rotation', 0)
        alignment = text_props.get('alignment', 'center')
        
        # Calculate font size
        text_font_size = text_props.get('font_size', font_size)
        if text_font_size is None:
            text_font_size = int(min(width, height) * 0.1)
        else:
            scale_factor = min(width, height) / 1000
            text_font_size = int(text_font_size * scale_factor)
        
        font = get_font(text_font_size)
        
        # Create temporary image for this text
        txt_img = Image.new('L', (width, height), 255)
        txt_draw = ImageDraw.Draw(txt_img)
        
        # Get text size
        text_bbox = txt_draw.textbbox((0, 0), text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        
        # Calculate absolute position
        abs_x = int(rel_x * width)
        abs_y = int(rel_y * height)
        
        # Adjust position based on alignment
        if alignment == 'center':
            abs_x -= text_width // 2
            abs_y -= text_height // 2
        elif alignment == 'right':
            abs_x -= text_width
            abs_y -= text_height // 2
        else:  # 'left'
            abs_y -= text_height // 2
            
        # Draw text
        txt_draw.text((abs_x, abs_y), text, fill=0, font=font)
        
        # Rotate if needed
        if rotation != 0:
            txt_img = txt_img.rotate(rotation, expand=False, fillcolor=255)
        
        # Composite onto main mask
        mask_img = Image.fromarray(np.minimum(np.array(mask_img), np.array(txt_img)))
    
    # Save mask
    if save_path is None:
        save_path = image_path.rsplit('.', 1)[0] + '_mask.png'
    mask_img.save(save_path)
    
    # Convert to numpy array
    mask = np.array(mask_img)
    
    print(f"Mask saved to: {save_path}")
    return mask

# Example usage:
if __name__ == "__main__":
    image_path = "doom.webp"
    
    # Define multiple texts with different properties
    texts = [
        {
            'text': 'Doom Eternal',
            'pos': (0.5, 0.3),        # Centered horizontally, at 30% from top
            'font_size': 200,
            'rotation': 0,
            'alignment': 'center'
        },
        {
            'text': 'Dark Ages',
            'pos': (0.5, 0.7),        # Centered horizontally, at 70% from top
            'font_size': 80,
            'rotation': 45,
            'alignment': 'center'
        },
        {
            'text': 'jsbdlkajdhhn',
            'pos': (0.5, 0.9),        # Bottom right corner
            'font_size': 100,
            'rotation': -45,          # Rotated 45 degrees counter-clockwise
            'alignment': 'right'
        }
    ]
    
    mask = create_text_mask(
        image_path=image_path,
        texts=texts,
        save_path='doom_mask.png'
    )

Mask saved to: doom_mask.png
